In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from django.contrib.auth import get_user_model
from django.db.models import Q
from events.models import *
from entities.models import *
User = get_user_model()

## 7. 서브쿼리

In [2]:
from django.db.models import Subquery

- User와 1대1로 연결된 UserParent 모델에서, UserParent이면서 User인 object를 조회하는 쿼리

In [3]:
users = User.objects.all()
queryset = UserParent.objects.filter(
    user_id__in=Subquery(
        users.values('id')
    )
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "events_userparent"."user_id", "events_userparent"."father_name", "events_userparent"."mother_name" FROM "events_userparent" WHERE "events_userparent"."user_id" IN (SELECT U0."id" FROM "auth_user" U0)


In [4]:
from django.db.models import OuterRef

- `OuterRef`를 이용하면 서브쿼리가 외부 쿼리의 정보를 참조할 수 있다.
- https://docs.djangoproject.com/en/4.0/ref/models/expressions/#referencing-columns-from-the-outer-queryset
- 가장 착한 Hero와 연결된 Category를 구하는 쿼리는 아래와 같다.

In [5]:
hero_qs = Hero.objects.filter(
    category=OuterRef("pk")
).order_by("-benevolence_factor")

category_qs = Category.objects.all().annotate(
    most_benevolent_hero = Subquery(
        hero_qs.values('name')[:1]
    )
)

SQL_query = str(category_qs.query)
print(SQL_query)

SELECT "entities_category"."id", "entities_category"."name", (SELECT U0."name" FROM "entities_hero" U0 WHERE U0."category_id" = "entities_category"."id" ORDER BY U0."benevolence_factor" DESC LIMIT 1) AS "most_benevolent_hero" FROM "entities_category"


## 8. F expression: 필드의 값을 서로 비교
- 우선 테스트할 오브젝트를 생성하자

```python
User.objects.create_user(
    email="shabda@example.com", 
    username="shabda", 
    first_name="Shabda", 
    last_name="Raaj"
)
User.objects.create_user(
    email="guido@example.com", 
    username="Guido", 
    first_name="Guido", 
    last_name="Guido"
)
```

- `F` expression은 필드와 필드를 추가적인 쿼리 수행 없이 비교할 수 있도록 한다.

In [6]:
queryset = User.objects.filter(
    last_name=F('first_name')
).values('username')
queryset

<QuerySet [{'username': 'admin'}, {'username': 'Guido'}]>

In [7]:
SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."username" FROM "auth_user" WHERE "auth_user"."last_name" = "auth_user"."first_name"


```python
User.objects.create_user(
    email='tim@example.com',
    username='Tim',
    first_name='Tim',
    last_name='Tam'
)
```

- annotate한 필드에 대해서도 `F` expression을 사용할 수 있다.
- `Substr`은 DB에서 str을 slicing한 후 검색할 수 있도록 하는 메소드이다.
- https://docs.djangoproject.com/en/4.0/ref/models/database-functions/#substr

In [8]:
from django.db.models.functions import Substr

queryset = User.objects.annotate(
    first=Substr('first_name', 1, 1),
    last=Substr('last_name', 1, 1),
).filter(
    first=F('last')
).values('first_name', 'last_name')
queryset

<QuerySet [{'first_name': '', 'last_name': ''}, {'first_name': 'sharukh', 'last_name': 'shohei'}, {'first_name': 'Guido', 'last_name': 'Guido'}, {'first_name': 'Tim', 'last_name': 'Tam'}]>

In [9]:
SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE SUBSTR("auth_user"."first_name", 1, 1) = SUBSTR("auth_user"."last_name", 1, 1)


## 9. FileField에 파일이 들어있지 않은 데이터?
- FileField는 응용수준에서의 구별이고 데이터베이스 수준에서는 모두 CharField와 동일한 방식으로 저장된다.
- 파일이 없는 데이터를 구하려면 다음과 같이 구하면 된다.

```python
no_files_objects = MyModel.objects.filter(
    Q(file='') |
    Q(file=None)
)
```